In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

Word Embedding and Neural Networks

In [3]:
df = pd.read_csv('NovoTrustPilotRatings.csv' , encoding='latin-1')
print(df.shape)


(3472, 9)


In [4]:
df.head()

,rating,rating title,Review text,Review date,Date of Experience,rating_procesed,Year of review,Year of experience,DIff in months
0,Rated 5 out of 5 stars,Great bank for businesses,"It was easy to set up, with no hassle like som...","Feb 15, 2024","December 01, 2023",5,2024,2023,2
1,Rated 3 out of 5 stars,Buggy when it matters most.,Unfortunately I???m probably going to figure o...,"Feb 19, 2024","February 18, 2024",3,2024,2024,0
2,Rated 5 out of 5 stars,Love digital banking,Love digital banking I keep now all my busines...,"Feb 1, 2024","February 01, 2024",5,2024,2024,0
3,Rated 3 out of 5 stars,Poor support and no wires,"A decent basic free business bank, but have re...","Feb 8, 2024","February 07, 2024",3,2024,2024,0
4,Rated 4 out of 5 stars,Really easy to set up,Really easy to set up. Works as intended most ...,"Jan 10, 2024","January 10, 2024",4,2024,2024,0


Only columns required for our purpose are rating title, Review text and rating processed

In [5]:
example = df['Review text'][50]
print(example)

Novo is awesome. Easy to navigate, fast transfers, beautifully designed. Novo was recommended to me by a fellow entrepreneur and I'm so glad I use them now too!


In [6]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [7]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [8]:

from tqdm.notebook import tqdm


In [9]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['Review text']
        roberta_result = polarity_scores_roberta(text)
        res[i] = roberta_result
    except RuntimeError:
        print(f'Broke for id {i}')
    except IndexError:
        print(f'index error at id {i}')

  0%|          | 0/3472 [00:00<?, ?it/s]

Broke for id 175
Broke for id 1046
index error at id 1511
Broke for id 1524
Broke for id 1720
Broke for id 1875
Broke for id 1945
Broke for id 2121
Broke for id 2155
Broke for id 2336
Broke for id 2355
Broke for id 2488
Broke for id 2606
Broke for id 2616
Broke for id 2618
Broke for id 2671
Broke for id 2795
Broke for id 2857


In [10]:
results_df = pd.DataFrame(res).T
results_df

,roberta_neg,roberta_neu,roberta_pos
0,0.001970,0.028291,0.969739
1,0.688426,0.247378,0.064196
2,0.001039,0.009252,0.989709
3,0.752764,0.224621,0.022615
4,0.107094,0.432379,0.460527
...,...,...,...
3467,0.972456,0.024977,0.002567
3468,0.859819,0.123987,0.016194
3469,0.807031,0.171405,0.021564
3470,0.790396,0.200038,0.009566
